In [1]:
!git clone https://github.com/amnatasneem/machine-learning-fall-2023-final-project.git

fatal: destination path 'machine-learning-fall-2023-final-project' already exists and is not an empty directory.


In [2]:
!pip install scikeras

In [3]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras

In [4]:
heart_data = pd.read_csv('/content/machine-learning-fall-2023-final-project/data/heart.csv', sep=",")
o2_data = pd.read_csv("/content/machine-learning-fall-2023-final-project/data/o2Saturation.csv")

In [5]:
heart_data.head(10)

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
5,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
6,56,0,1,140,294,0,0,153,0,1.3,1,0,2,1
7,44,1,1,120,263,0,1,173,0,0.0,2,0,3,1
8,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1
9,57,1,2,150,168,0,1,174,0,1.6,2,0,2,1


In [6]:
o2_data.head(10)

,98.6
0,98.6
1,98.6
2,98.6
3,98.1
4,97.5
5,97.5
6,97.5
7,97.5
8,97.5
9,97.5


In [7]:
# Classification: heart attack prediction
X_classification = heart_data.drop(heart_data.columns[13], axis=1)
y_classification = heart_data[heart_data.columns[13]]

# Split the data (train and test)
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_classification, y_classification, test_size=0.2, random_state=50
)

In [8]:
# Standardize features
scaler = StandardScaler()
X_train_class_scaled = scaler.fit_transform(X_train_class)
X_test_class_scaled = scaler.transform(X_test_class)

In [9]:
# Logistic regression
log_reg_model = LogisticRegression()
log_reg_model.fit(X_train_class_scaled, y_train_class)
y_pred_labels = log_reg_model.predict(X_test_class_scaled)

In [10]:
# Print classification results
print("Logistic Regression - Classification Results: ")
print(classification_report(y_test_class, y_pred_labels))

Logistic Regression - Classification Results: 
              precision    recall  f1-score   support

           0       0.80      0.64      0.71        25
           1       0.78      0.89      0.83        36

    accuracy                           0.79        61
   macro avg       0.79      0.76      0.77        61
weighted avg       0.79      0.79      0.78        61



In [11]:
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
# Neural network
def create_nn_model():
    model = Sequential()
    model.add(Dense(10, input_dim=X_train_class_scaled.shape[1], activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

nn_model = KerasClassifier(model=create_nn_model, epochs=10, batch_size=32, verbose=0)
nn_model.fit(X_train_class_scaled, y_train_class)
y_pred_nn = nn_model.predict(X_test_class_scaled)

In [12]:
# Build Sequential model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_dim=X_train_class_scaled.shape[1], activation="relu"),
    tf.keras.layers.Dense(16, input_dim=X_train_class_scaled.shape[1], activation="relu"),
    tf.keras.layers.Dense(32, input_dim=X_train_class_scaled.shape[1], activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train model
model.fit(X_train_class_scaled, y_train_class, epochs=10, batch_size=32, verbose=0)

# Evaluate model on test set
y_pred_prob = model.predict(X_test_class_scaled)
y_pred_labels = (y_pred_prob > 0.5).astype(int)

# Print results
print("Neural Network - Classification Results:")
print(confusion_matrix(y_test_class, y_pred_labels))
print(classification_report(y_test_class, y_pred_labels))

2/2 [==============================] - 0s 7ms/step
Neural Network - Classification Results:
[[18  7]
 [ 8 28]]
              precision    recall  f1-score   support

           0       0.69      0.72      0.71        25
           1       0.80      0.78      0.79        36

    accuracy                           0.75        61
   macro avg       0.75      0.75      0.75        61
weighted avg       0.76      0.75      0.75        61



In [13]:
# Regression: O2 saturation prediction
X_regression = o2_data.drop(o2_data.columns[0], axis=1)
y_regression = o2_data[o2_data.columns[0]]

In [14]:
# Split the data (train and test)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_regression, y_regression, test_size=0.2, random_state=50
)